# **Numerical Data 다루기**

## 4.1 특성 scale 바꾸기

* 수치형 특성이 두 값의 범위 안에 놓이도록 scale을 바꿔야 함
* sklearn의 MinMaxScale를 사용해 특성배열의 스케일을 조정

In [5]:
import numpy as np
from sklearn import preprocessing

feature = np.array([
    [-500.5],
    [-100.1],
    [0],
    [100.1],
    [900.9]
]) 
#특성을 생성함

minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
#scaler 객체 생성

scaled_feature = minmax_scaler.fit_transform(feature)
#특성의 scale을 변환

scaled_feature
#scale이 0~1로 바뀜

array([[0.        ],
       [0.28571429],
       [0.35714286],
       [0.42857143],
       [1.        ]])

In [ ]:
import numpy as np
from sklearn import preprocessing

In [10]:
feature

array([[-500.5],
       [-100.1],
       [   0. ],
       [ 100.1],
       [ 900.9]])

In [8]:
feature.dtype

dtype('float64')

In [9]:
scaled_feature.dtype

dtype('float64')

스케일 조정은 머신러닝의 전처리 작업이다. 대부분 알고리즘은 모든 특성이 동일한 scale을 갖는다고 가정한다.   
일반적으로 0~1이나 -1~1 사이이다.

## Min-max scaling
*  특성의 최솟값과 최댓값을 사용해 일정 범위 안으로 값을 조정한다. 
* 계산 과정
$$
x_i^` = \frac{x_i - min(x)}{max(x) - min(x)}
$$
* x는 feature vector, $x_i$는 스케일이 바뀐 원소

* **fit**과 **transform** method, **fit_transform**method   
fit: 특성의 최댓값과 최솟값을 계산한 후   
trasform: 특성의 스케일을 조정

**주의!! train set과 test set의 scale을 따로 조정하면 안됨   
train set의 scale을 조정하고자 하면 test set을 또한 같이 변환해야함**

In [15]:
preprocessing.MinMaxScaler()

MinMaxScaler(copy=True, feature_range=(0, 1))

* wrong : feature의 처음 세개를 train set, 나머지 두개를 test set이라 가정하고 독립적으로 변환

In [17]:
#train set
preprocessing.MinMaxScaler().fit_transform(feature[:3])

array([[0. ],
       [0.8],
       [1. ]])

In [18]:
#test set
preprocessing.MinMaxScaler().fit_transform(feature[3:])

array([[0.],
       [1.]])

결론: train set의 0이 1, tset set의 900.9가 1로 변환.   
데이터가 다른 스케일로 변환되면 train set에서 학습한 모델을 test set에 사용할 수 없음

* train set에서 학습한 변환기로 test set 학습하기

In [24]:
scaler=preprocessing.MinMaxScaler().fit(feature[:3])
#동시에 train, test set scale 변환
scaler.transform(feature[:3])

array([[0. ],
       [0.8],
       [1. ]])

In [23]:
scaler.transform(feature[3:])

array([[1.2],
       [2.8]])

결론 : train set을 학습한 변환기 객체를 사용하면 원본 데이터셋과 동일한 비율로 test set을 반환

***

## 4.2 특성 표준화하기

* 특성을 평균이 0이고 표준편차가 1이 되도록 변환해야 함
* 사이킷런의 StandardScaler을 사용해 두 변환을 모두 수행

In [10]:
import numpy as np
from sklearn import preprocessing

x = np.array([
    [-1000.1],
    [-200.2],
    [500.5],
    [600.6],
    [9000.9]
])

scaler = preprocessing.StandardScaler()
# 변환기 객체 생성

standardized = scaler.fit_transform(x)
#특성 변환

standardized

array([[-0.76058269],
       [-0.54177196],
       [-0.35009716],
       [-0.32271504],
       [ 1.97516685]])

### 표준 정규분포로 근사하는 scalering 방식

* 데이터의 평균$\bar x$가 0이고, 표준편차$\sigma$가 1이되도록 변환
* 변환 과정
$$
x_i^` = \frac{x_i - \bar x}{\sigma}
$$
* $x_I^`$는 $x_i$의 표준화된 형태
* 변환된 feature은 원본 값이 특성 평균에서 몇 표준편차만큼 떨어져 있는지로 표현함

주성분 분석은 표준화, 신경망은 최소-최대 scaling이 권장

예시)

In [4]:
print("평균 {}".format(round(standardized.mean()))) #round 반올림 연산 실행
print("표준편차: {}".format(standardized.std()))

평균 0.0
표준편차: 1.0


#### Robustscaler

* 데이터에 이상치가 많다면 특성의 평균과 표준편차에 영향을 미치기 때문에 표준화에 부정적 영향을 미침*   
    따라서 이 경우 중간값과 사분위 범위를 사용해 특성의 스케일을 조정해야 함. (sklearn의 RobustScalar)
* 데이터를 오름차순으로 나열했을 때 75%에 위치한 값과 25%에 위치한 값의 차를 사분위범위(IQR)이라 부름

**데이터에서 중간값을 빼고 IQR로 나눔**

In [6]:
# create scaler
robust_scaler = preprocessing.RobustScaler()

# transform feature
robust_scaler.fit_transform(x)

array([[-1.87387612],
       [-0.875     ],
       [ 0.        ],
       [ 0.125     ],
       [10.61488511]])

In [8]:
iqr= x[3]-x[1]
(x-np.median(x))/iqr


array([[-1.87387612],
       [-0.875     ],
       [ 0.        ],
       [ 0.125     ],
       [10.61488511]])

#### QuantileTransformer

* 훈련데이터를 1000개의 분위로 나눠 0~1사이에 고르게 분포시킴으로써 이상치로 인한 영향을 줄임
* x는 5개의 샘플을 가지므로 0%,25%, 50%, 75%, 100%의 위치에 할당

In [11]:
preprocessing.QuantileTransformer().fit_transform(x)

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (5). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


array([[0.  ],
       [0.25],
       [0.5 ],
       [0.75],
       [1.  ]])

***

## 4.3 정규화하기

* sample의 특성값을 전체 길이 1인 단위 norm이 되도록 변환
* norm매개변수와 Normalizer 클래스 사용

In [13]:
import numpy as np
from sklearn.preprocessing import Normalizer

# 특성 행렬 생성
features = np.array([
    [0.5, 0.5],
    [1.1, 3.4],
    [1.5, 20.2],
    [1.63, 34.4],
    [10.9, 3.3]
])

# 변환기 객체 생성
normalizer = Normalizer(norm="l2")

# 특성 행렬 변환
normalizer.transform(features)

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

#### Normalizer

* 단위norm(길이의 합이 1)이 되도록 개별 샘플의 값을 변환
* 3가지 norm옵션을 제공

    1. L2 norm: 기본값, 유클리드 norm 
    2. L1 norm: 맨해튼 norm, 샘플 특성값의 합을 1로 만든다   
    http://hleecaster.com/ml-distance-formula/
    3. "max": 각 행의 최댓값으로 행의 값을 나눔 

In [17]:
feature_l2_norm= Normalizer(norm="l2").transform(features)

feature_l2_norm

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

In [18]:
features / np.sqrt(np.sum(np.square(features), axis=1, keepdims=True))

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

In [19]:
feature_l1_norm= Normalizer(norm="l1").transform(features)

print(feature_l1_norm)

print("첫번째 샘플의 합: ", feature_l1_norm[0,0]+feature_l1_norm[0,1])

[[0.5        0.5       ]
 [0.24444444 0.75555556]
 [0.06912442 0.93087558]
 [0.04524008 0.95475992]
 [0.76760563 0.23239437]]
첫번째 샘플의 합:  1.0


In [20]:
features / np.sum(np.abs(features), axis=1, keepdims=True) #abs()절댓값 함수

array([[0.5       , 0.5       ],
       [0.24444444, 0.75555556],
       [0.06912442, 0.93087558],
       [0.04524008, 0.95475992],
       [0.76760563, 0.23239437]])

In [21]:
Normalizer(norm="max").transform(features)

array([[1.        , 1.        ],
       [0.32352941, 1.        ],
       [0.07425743, 1.        ],
       [0.04738372, 1.        ],
       [1.        , 0.30275229]])

***

## 4.4 다항 특성과 교차항 특성 생성하기

### PolynomialFeatures(degree= , include_bias= )

* 지정된 차수 이하인 feature의 모든 다항식 조합으로 구성된 새로운 행렬을 생성
* degree매개변수가 다항식의 최대찿수를 결정
    - ex)degree=2는 제곱까지 새로운 특성을 만듬   
    degree=3은 제곱과 3제곱까지 새로운 특성을 만듬


In [17]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

# 특성 행렬 생성
features = np.array([
    [2, 3],
    [2, 3],
    [2, 3]])

polynomial_interaction = PolynomialFeatures(degree=2, include_bias=False)
polynomial_interaction.fit_transform(features)

array([[2., 3., 4., 6., 9.],
       [2., 3., 4., 6., 9.],
       [2., 3., 4., 6., 9.]])

* 기본적으로 교차항을 포함한다
    - interaction_only를 True로 지정하면 교차항 특성만 만들 수 있음

In [18]:
polynomial_interaction = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
polynomial_interaction.fit_transform(features)

array([[2., 3., 6.],
       [2., 3., 6.],
       [2., 3., 6.]])

* include_bias 매개변수: 기본값은 True, 변환된 특성에 상수항 1을 추가한다.

In [22]:
polynomial_bias = PolynomialFeatures(degree=2, include_bias=True).fit(features)
polynomial_bias.transform(features)

array([[1., 2., 3., 4., 6., 9.],
       [1., 2., 3., 4., 6., 9.],
       [1., 2., 3., 4., 6., 9.]])

* get_feature_names() : 특성변환 식을 이름으로 반환한다.

In [23]:
polynomial_bias.get_feature_names()

['1', 'x0', 'x1', 'x0^2', 'x0 x1', 'x1^2']

***

### 4.5 특성 변환하기

### sklearn의 FunctionTransformse

* 일련의 특성에서 어떤 함수를 적용할 수 있음

In [23]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

# 특성 행렬 생성
features = np.array([
    [2, 3],
    [2, 3],
    [2, 3]])

def add_ten(x):
    return x+10

ten_transformer=FunctionTransformer(add_ten)

ten_transformer.transform(features)

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\preprocessing\_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


array([[12, 13],
       [12, 13],
       [12, 13]])

* validate매개변수 : Ture-입력값이 2차원 배열인지 확인하고 아닐 경우 예외 발생   
                    False(기본값)- 1차원 배열에도 적용

In [24]:
FunctionTransformer(add_ten, validate=False).transform(np.array([1,2,3]))

array([11, 12, 13])

### pandas의 apply()

* FunctionTransformser와 동일

In [27]:
import pandas as pd

df= pd.DataFrame(features, columns=["feature_1","feature_2"])

df.apply(add_ten)

,feature_1,feature_2
0,12,13
1,12,13
2,12,13


### ColumnTransformer

* 특성배열이나 데이터프레임의 열마다 다른 변환을 적용할 수 있음

In [28]:
from sklearn.compose import ColumnTransformer

def add_hundred(x):
    return x + 100

#(이름, 변환기, 열 리스트)로 구성된 튜플의 리스트를 ColumnTransformer에 전달
ct = ColumnTransformer(
[
    ("add_ten",FunctionTransformer(add_ten, validate=True),["feature_1"]),
    ("add_hundred",FunctionTransformer(add_hundred, validate=True),["feature_2"])
])

ct.fit_transform(df)

array([[ 12, 103],
       [ 12, 103],
       [ 12, 103]])

***

### 4.6 Outlier 감지하기

* data가 정규분포를 따른다고 가정하고 data를 둘러싼 타원을 그림.   
    타원 안의 샘플을 정상치, rable 1로 분류   
    타원 밖의 샘플은 이상치, rable -1로 분류

### EllipticEnvelope(contamination= )

* 정규분포 집합에서 특이치를 탐지하는 객체
* contamination: 데이터가 얼마나 깨끗한지 추측하는 것
    - 데이터에 이상치가 적다면 contamination를 작게 지정, 이상치 많으면 많게 지정

In [37]:
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.datasets import make_blobs


features, _ = make_blobs(n_samples=10,
                         n_features=2,
                        centers=1,
                        random_state=1)
#

features[0,0]=10000
features[0,1]=10000

outlier_detector = EllipticEnvelope(contamination=.1)
outlier_detector.fit(features)
outlier_detector.predict(features)

array([-1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

### 사분위범위(IQR)을 사용해 개별특성에서 극단적인 값 구별하기

* iqr은 데이터의 대부분이 퍼져있는 곳으로 생각할 수 있다.
    이상치는 데이터가 집중되 있는 곳에서 멀리 떨어진 샘플이다.
* 보통 이상치는 1사분위보다 1.5iqr이상 작은 값이나 3사분위보다 1.5iqr 큰 값으로 정의

In [38]:
feature=features[:,0]

def indicies_of_outliers(x):
    q1, q3=np.percentile(x,[25,75])
    iqr=q3-q1
    lower_bound=q1-(iqr*1.5)
    upper_bound=q3+(iqr*1.5)
    return np.where((x>upper_bound)|(x<lower_bound))

indicies_of_outliers(feature)

(array([0], dtype=int64),)

***

### 4.7 Outlier 다루기

### 이상치를 다루는 3가지 전략

1. 이상치 삭제하기

In [31]:
import pandas as pd

houses =pd.DataFrame()
houses['Price']=[534433,392333,293222,4322032]
houses['Bathrooms']=[2,3.5,2,116]
houses['Square_Feet']=[1500,2500,1500,40000]

houses[houses['Bathrooms']<20]

,Price,Bathrooms,Square_Feet
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500


2. 이상치로 표시하고 이를 특성의 하나로 포함

In [32]:
import numpy as np

houses["Outlier"]=np.where(houses['Bathrooms']<20,0,1) 
#조건에 따라 x 또는 y에서 선택한 요소를 반환합니다.
houses

,Price,Bathrooms,Square_Feet,Outlier
0,534433,2.0,1500,0
1,392333,3.5,2500,0
2,293222,2.0,1500,0
3,4322032,116.0,40000,1


3. 이상치의 영향이 줄어들도록 특성 변환

In [33]:
houses["Log_of_Square_Feet"]=[np.log(x) for x in houses["Square_Feet"]]
#Square_Feet 값에 로그 취하기
houses

,Price,Bathrooms,Square_Feet,Outlier,Log_of_Square_Feet
0,534433,2.0,1500,0,7.313220
1,392333,3.5,2500,0,7.824046
2,293222,2.0,1500,0,7.313220
3,4322032,116.0,40000,1,10.596635


주의! 이상치가 평균과 분산에 영향을 끼치기 때문에 이상치가 있다면 표준화가 적절하지 않다.
RobustSclaer와 같이 이상치애 민감하지 않은 스켕일링 방법을 사용하기

***

### 4.8 특성 이산화하기(discretization)

In [43]:
import numpy as np
from sklearn.preprocessing import Binarizer

age = np.array([[6],
              [12],
              [20],
              [36],
              [65]])

binarizer = Binarizer(18)
binarizer.fit_transform(age)


array([[0],
       [0],
       [1],
       [1],
       [1]])

In [44]:
np.digitize(age, bins=[20,30,64])

array([[0],
       [0],
       [1],
       [2],
       [3]], dtype=int64)

In [45]:
np.digitize(age, bins=[20,30,40], right=True)

array([[0],
       [0],
       [0],
       [2],
       [3]], dtype=int64)

In [46]:
np.digitize(age, bins=[18])

array([[0],
       [0],
       [1],
       [1],
       [1]], dtype=int64)

***

### 4.9 군집으로 sample을 group으로 묶기

### K-평균 군집

* 각 군집은 하나의 중심(centroid)을 가진다
* 각 개체는 가장 가까운 중심에 할당되며, 같은 중심에 할당된 개체들이 모여 하나의 군집을 형성한다

* **총 N 개의 데이터로 D 개의 차원을 가지며 K 개의 클러스터로 나눌 예정이다.
    최종 목표는 모든 샘플 데이터를 K 개의 클러스터에 각각 배정하는 것이다.**
https://ratsgo.github.io/machine%20learning/2017/04/19/KC/

#### 작동방식

1. k개의 클러스터 중심 포인트를 랜덤한 위치에 만든다
2. 각 샘플에 대해
    - 각 샘플과 k개의 중심 포인트 사이 거리를 계산한다
    - 샘플을 가장 가까운 중심 포인트의 클러스터에 할당한다
3. 중심 포인트를 해당하는 클러스터의 평균(중심)으로 이동
4. 더 이상 샘플의 클러스터 소속이 바뀌지 않을 때까지 단계2와 단계3반복

In [1]:

import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

features, _ = make_blobs(n_samples = 50,
                         n_features = 2,
                         centers = 3,
                         random_state = 1)

df = pd.DataFrame(features, columns=["feature_1", "feature_2"])

# K-MEAN CLUSTER 모델울 만듬
clusterer = KMeans(3, random_state=0)

# 모델 훈련
clusterer.fit(features)

# 그줍 소속을 예측
df['group'] = clusterer.predict(features)

df.head()

,feature_1,feature_2,group
0,-9.877554,-3.336145,0
1,-7.287210,-8.353986,2
2,-6.943061,-7.023744,2
3,-7.440167,-8.791959,2
4,-6.641388,-8.075888,2


***

### 4.10 누락된 값을 가진 sample 삭제하기

### numpy에선 **~연산자**로

In [34]:
import numpy as np

features = np.array([
    [1.1, 11.1],
    [2.2, 22.2],
    [3.3, 33.3],
    [np.nan, 55]
])

# ^로 누락되지않은 샘플만 남김
features[~np.isnan(features).any(axis=1)]

array([[ 1.1, 11.1],
       [ 2.2, 22.2],
       [ 3.3, 33.3]])

### pandas의 dropna()

In [35]:
import pandas as pd
df = pd.DataFrame(features, columns=["feature_1", "feature_2"])

#누락된 샘플만 제거
df.dropna()

,feature_1,feature_2
0,1.1,11.1
1,2.2,22.2
2,3.3,33.3


누락된 값의 원인에 따라 샘플 삭제는 데이터의 편향을 늘린다는 사실 주의

### 누락된 data 종류
1. 완전히 random하게 누락
    값이 누락될 확률이 모든 것에 독립적
2. random하게 누락
    값이 누락될 확률이 랜덤하지 않고 다른 특성에서 얻은 정보에 의존
    ex) 자녀 유무 항목이 결혼 여부 특성에 의존 
3. random하지 않게 누락
    값이 누락될 확률이 특성에서 잡지 못한 정보에 의존
    ex)자녀 유무 항목에 결혼 여부 특성이 포함되지 않은 경우

***

### 4.11 누락된 값 채우기

### k-최근접 이웃(KNN) 알고리즘

* 데이터의 양이 작을 때 새로운 데이터를 입력받으면 주변 가장 가까이 있는 sample들을 중심으로 데이터 종류를 정해주는 알고리즘
* k매개변수 : 탐색할 이웃 수, 일반적으로 총데이터의 제곱근값 사용

In [39]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from fancyimpute import KNN

features, _ = make_blobs(n_samples = 1000,
                        n_features = 2,
                        random_state = 1)

# 특성 표준화하기
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)

# 첫번쨰 샘플의 첫번째 특성 삭제
true_value = standardized_features[0, 0]
standardized_features[0,0] = np.nan

#특성 행렬에 있는 누락된 값을 예측, k: 탐색할 이웃 수
features_knn= KNN(k=5,verbose=0).fit_transform(standardized_features)

print("True Value: ",true_value)
print("Imputed Value: ",feautres_knn[0,0])



ModuleNotFoundError: No module named 'fancyimpute'

### Imputer 모듈

* 특성의 평균, 중간값, 최빈값으로 누락된 값을 채울 수 있음
    - strategy 매개변수 : 'mean','median','most_frequent',
        'constant'(full_value매개변수에 지정된 값으로 대체)

In [ ]:
from sklearn.preprocessing import Imputer

# create imputer
mean_imputer = Imputer(strategy="mean", axis=0)

# impute values
feautres_mean_imputed = mean_imputer.fit_transform(features)

# compare true and imputed values
print("True Value: {}".format(true_value))
print("Imputed Value: {}".format(feautres_mean_imputed[0,0]))